In [1]:
import os, random, logging
import utils.auxua as aux
import utils.watch as watch
import utils.tf_formh5 as tf_formh5
import numpy as np

''' tf log level & neptune'''
tf_formh5.set_tf_loglevel(logging.ERROR)

cudatoolkit               10.1.243            h8cb64d8_11    conda-forge
cudnn                     7.6.5.32             hc0a50b0_1    conda-forge
neptune-tensorflow-keras  2.1.0              pyhd8ed1ab_0    conda-forge
numpy                     1.19.5           py38hc896f84_4  
numpy-base                1.19.5           py38h21a3de8_4  
tensorflow                2.2.0           gpu_py38hb782248_0    anaconda
tensorflow-base           2.2.0           gpu_py38h83e3d50_0    anaconda
tensorflow-estimator      2.6.0            py38h709712a_0    conda-forge
tensorflow-gpu            2.2.0                h0d30ee6_0    anaconda

Num GPUs Available:  4


In [2]:
'''get_results_from_txt'''

res_list_full,res_list_max,res_list_fn,res_list_path,res_list_labels,res_path,res_model_fn,res_model_fn_strap =\
aux.get_results_from_txt(aux.RSLT_PATH,printt=True)

res_list={
    'full':res_list_full,
    'max':res_list_max,
    'videoname':res_list_fn,
    'videopath':res_list_path,
    'labels':res_list_labels,
    'txt_path':res_path,
    'txt_modelname':res_model_fn,
    'txt_modelname_strap':res_model_fn_strap        
}

for i in range(len(res_list['txt_modelname'])):print(i,res_list['txt_modelname'][i])

#res_list_1 = aux.get_results_from_txt("/raid/DATASETS/.zuble/vigia/zu++/model/rslt/1671124367.446378_3gelu_adamamsgrad_0_4000_weights.txt")

FOLDER
sort /raid/DATASETS/.zuble/vigia/zu++/model/rslt/1679349568.3157873_leakyrelu_adamamsgrad_0_4000_weights-1_1000.txt -o /raid/DATASETS/.zuble/vigia/zu++/model/rslt/1679349568.3157873_leakyrelu_adamamsgrad_0_4000_weights-1_1000.txt
sort /raid/DATASETS/.zuble/vigia/zu++/model/rslt/1677498953.2416248_relu_sgd_0_4000_weights-2_1000.txt -o /raid/DATASETS/.zuble/vigia/zu++/model/rslt/1677498953.2416248_relu_sgd_0_4000_weights-2_1000.txt
sort /raid/DATASETS/.zuble/vigia/zu++/model/rslt/1671124357.725051_3gelu_adam_0_4000_weights.txt -o /raid/DATASETS/.zuble/vigia/zu++/model/rslt/1671124357.725051_3gelu_adam_0_4000_weights.txt
sort /raid/DATASETS/.zuble/vigia/zu++/model/rslt/1677002476.5250552_relu_adam_0_2000_weights.txt -o /raid/DATASETS/.zuble/vigia/zu++/model/rslt/1677002476.5250552_relu_adam_0_2000_weights.txt
sort /raid/DATASETS/.zuble/vigia/zu++/model/rslt/1676724247.3618405_leakyrelu_adam_0_2000_weights.txt -o /raid/DATASETS/.zuble/vigia/zu++/model/rslt/1676724247.3618405_leakyre

In [3]:
'''
TYPE OF EVENT                                 
    B1 : FIGHT                                  _______
    B2 : SHOOTING                              | LABEL |
    B4 : RIOT                          ________|_0_|_1_|
    B5 : ABUSE                        |Anom| O |TN | FN|              
    B6 : CAR ACCIDENT                 |Scor|_1_|FP_|_TP|        
    G  : EXPLOSION                       
                                  
watch_this : 
    
    'N'        : All Normal       
    'A'        : All Anomalies                
    'A + type' : Filter  
         
    'FP'        : LABEL 0 | AS 1
    'TN'        : LABEL 0 | AS 0
    'FN + type' : LABEL 1 | AS 0
    'TP + type' : LABEL 1 | AS 1
        if type BG :groups all anomalies      
'''

watch.init_watch_reslist(res_list,11,watch_this=['A'])

as_total from /raid/DATASETS/.zuble/vigia/zu++/model/rslt/1679349568.3157873_leakyrelu_adamamsgrad_0_4000_weights-2_3000.txt

get_fn_index_from_list  1679349568.3157873_leakyrelu_adamamsgrad_0_4000_weights-2_3000

ANOMALIES 
  All         500 
  AB1 FIGHT   126 
  AB2 SHOOT   104 
  AB4 RIOT    101 
  AB5 ABUSE   11 
  AB6 CARACC  106 
  AG EXPLOS   103 

FALSE NEGATIVES (LABEL1 / AS0) 
  FN.B1 FIGHT   22 
  FN.B2 SHOOT   15 
  FN.B4 RIOT    25 
  FN.B5 ABUSE   7 
  FN.B6 CARACC  5 
  FN.G EXPLOS   13 
  FN.B+G        81 

TRUE POSITIVES (LABEL1 / AS1) 
  TP.B1 FIGHT   98 
  TP.B2 SHOOT   71 
  TP.B4 RIOT    76 
  TP.B5 ABUSE   3 
  TP.B6 CARACC  92 
  TP.G EXPLOS   79 
  TP.B+G        419 

NORMALS 
  All         300 

FALSE POSITIVES (LABEL0 / AS1) 
  FP   51 

TRUE NEGATIVES (LABEL0 / AS0) 
  TN   249 

set to watch this ['A']

 A
8
9
10
26
27
29
30
31
32
33
34
36
37
38
39
41
42
44
46
47
49
50
52
53
54
55
57
58
59
60
61
62
63
64
66
68
70
71
72
74
75
76
78
79
82
83
84
85
86
88
89
90
